In [3]:
'''
English:

Use this function to connect to Firestore

SDK_Path: The Path of the SDK file used to connect to Firestore

Português:

Use essa função para se conectar no Firestore
SDK_Path: O caminho do arquivo SDK usado para se conectar no Firestore


'''

def FirestoreConnector(SDK_Path):
    import firebase_admin
    from firebase_admin import credentials       
    cred = credentials.Certificate(SDK_Path)
    app = firebase_admin.initialize_app(cred)
    return app

In [10]:
'''
English:

Use this function to extract data from Firestore to a Pandas Dataframe

collection_name: The Collection you want to extract 
app: Firestore Connector (Use FirestoreConnector to get this variable)
batch_size: Batch of date you'll load per time from your connection
cursor: The "document number" from query

Português:

Use essa função para extrair dados do Firestore para um Dataframe Pandas

collection_name: A Collection que você quer extrair
app: Conector do Firestore (Use a função FirestoreConnector para conseguir esta variável)
batch_size: Lote de dados que você vai carregar por vez de sua conexão
cursor: O "número documento atual" da query.

'''

def FirestoreExtractor(collection_name, app, batch_size=4000, cursor=None):
    import pandas as pd
    import firebase_admin
    from firebase_admin import credentials   
    import pandas as pd
    import json
    from firebase_admin import firestore
    import datetime
    
    db = firestore.client(app)
    print('Connected to Firestore')

    def iterate(collection_name, batch_size, cursor):
        query = db.collection(collection_name).limit(batch_size).order_by('__name__')
        if cursor:
            query = query.start_after(cursor)

        for doc in query.stream():
            yield doc

        if 'doc' in locals():
            yield from iterate(collection_name, batch_size, doc)
    print('Creating Dataframe')
    items = list(map(lambda x: {**x.to_dict(), 'id': x.id}, iterate(collection_name, batch_size, cursor)))
    df = pd.DataFrame(items)
    df.set_index('id', inplace=True)
    
    return df